#Weather Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import missingno as msno
%matplotlib inline

years = ("2016", "2017")

# Data for filtering
airports = ("ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO")
fields = ("windspeedKmph",
          "winddirDegree",
          "weatherCode",
          "precipMM",
          "visibility",
          "pressure",
          "cloudcover",
          "DewPointF",
          "WindGustKmph",
          "tempF",
          "WindChillF",
          "humidity",
          "date",
          "time")


In [ ]:
# Loading one json file to analyze its structure
data = pd.read_json("/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_1.json")
print(type(data))
data

<class 'pandas.core.frame.DataFrame'>


,data
request,"[{'query': 'ATL, Hartsfield-Jackson Atlanta In..."
weather,"[{'mintempC': '5', 'maxtempF': '51', 'sunHour'..."


In [ ]:
# Isolating the weather data and examining its columns
wd = data["data"]["weather"]
print(type(wd), end="\n\n")
wd_df = (pd.DataFrame(wd))
print(wd_df.columns, end="\n\n")
print(wd_df.shape, end="\n\n")
print(wd_df.index, end="\n\n")
wd_df.head()

<class 'list'>

Index(['mintempC', 'maxtempF', 'sunHour', 'mintempF', 'maxtempC', 'hourly',
       'totalSnow_cm', 'date', 'astronomy', 'uvIndex'],
      dtype='object')

(31, 10)

RangeIndex(start=0, stop=31, step=1)



,mintempC,maxtempF,sunHour,mintempF,maxtempC,hourly,totalSnow_cm,date,astronomy,uvIndex
0,5,51,7.5,42,11,"[{'windspeedKmph': '11', 'FeelsLikeF': '46', '...",0.0,2016-01-01,"[{'moon_phase': 'Waning Gibbous', 'moonrise': ...",0
1,2,48,9.0,35,9,"[{'windspeedKmph': '15', 'FeelsLikeF': '34', '...",0.0,2016-01-02,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
2,3,51,7.5,38,11,"[{'windspeedKmph': '10', 'FeelsLikeF': '37', '...",0.0,2016-01-03,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
3,-1,46,9.5,31,8,"[{'windspeedKmph': '13', 'FeelsLikeF': '37', '...",0.0,2016-01-04,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
4,1,43,10.0,33,6,"[{'windspeedKmph': '12', 'FeelsLikeF': '28', '...",0.0,2016-01-05,"[{'moon_phase': 'Waning Crescent', 'moonrise':...",0


In [ ]:
# columns hourly has a lot of useful nested fields that are of interest so the nested data needs flattening
wd_df = pd.json_normalize(data=wd)
print(wd_df.columns, end="\n\n")
print(wd_df.shape, end="\n\n")
print(wd_df.index, end="\n\n")
# print(wd_df.iloc[0]["hourly"])
wd_df.head()

Index(['mintempC', 'maxtempF', 'sunHour', 'mintempF', 'maxtempC', 'hourly',
       'totalSnow_cm', 'date', 'astronomy', 'uvIndex'],
      dtype='object')

(31, 10)

RangeIndex(start=0, stop=31, step=1)



,mintempC,maxtempF,sunHour,mintempF,maxtempC,hourly,totalSnow_cm,date,astronomy,uvIndex
0,5,51,7.5,42,11,"[{'windspeedKmph': '11', 'FeelsLikeF': '46', '...",0.0,2016-01-01,"[{'moon_phase': 'Waning Gibbous', 'moonrise': ...",0
1,2,48,9.0,35,9,"[{'windspeedKmph': '15', 'FeelsLikeF': '34', '...",0.0,2016-01-02,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
2,3,51,7.5,38,11,"[{'windspeedKmph': '10', 'FeelsLikeF': '37', '...",0.0,2016-01-03,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
3,-1,46,9.5,31,8,"[{'windspeedKmph': '13', 'FeelsLikeF': '37', '...",0.0,2016-01-04,"[{'moon_phase': 'Last Quarter', 'moonrise': '0...",0
4,1,43,10.0,33,6,"[{'windspeedKmph': '12', 'FeelsLikeF': '28', '...",0.0,2016-01-05,"[{'moon_phase': 'Waning Crescent', 'moonrise':...",0


In [ ]:
# flattening the data by one more column hourly retaining the date column
wd_df = pd.json_normalize(data=wd, record_path="hourly", meta="date")
col1 = set(wd_df.columns)
wd_df = wd_df[list(fields)]
col2 = wd_df.columns
print(col1.intersection(col2))
wd_df

{'WindGustKmph', 'winddirDegree', 'windspeedKmph', 'cloudcover', 'weatherCode', 'visibility', 'tempF', 'precipMM', 'DewPointF', 'date', 'time', 'pressure', 'humidity', 'WindChillF'}


,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,13,169,122,0.0,10,1015,96,54,20,60,60,82,2016-01-31,1900
740,13,165,122,0.0,10,1015,95,54,20,60,59,82,2016-01-31,2000
741,12,162,122,0.0,10,1015,95,54,20,59,59,83,2016-01-31,2100
742,13,164,122,0.0,10,1015,92,54,21,59,58,83,2016-01-31,2200


In [ ]:
#Loading data
weather_data = list()
# Iterating over the files and performing data handling
for city in airports:
    for year in years:
        for month in range(1, 12+1):
            file = f"/content/drive/MyDrive/Data (1)/Weather_Data/{city}/{year}_{month}.json"


            df = pd.read_json(file)
            wd = df["data"]["weather"]
            wd_df = pd.json_normalize(data=wd)
            wd_df = pd.json_normalize(data=wd, record_path="hourly", meta="date")
            wd_df = wd_df[list(fields)]
            wd_df["airport"] = city
            weather_data.append(wd_df)
            print(file)

/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_2.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2016_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2017_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2017_2.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2017_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/ATL/2017_4.json
/content/drive/MyDriv

<ipython-input-82-e96f9173de67>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_df["airport"] = city


/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/CLT/2017_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_2.json


<ipython-input-82-e96f9173de67>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_df["airport"] = city


/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/DEN/2016_11.json


...

/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/ORD/2017_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_2.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2016_9.json
/content/drive/MyDriv

<ipython-input-82-e96f9173de67>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_df["airport"] = city


/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2017_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2017_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2017_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2017_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/PHX/2017_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_2.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2016_11.json
/content/drive/MyDr

<ipython-input-82-e96f9173de67>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_df["airport"] = city


/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2017_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2017_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/SEA/2017_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_1.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_2.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_3.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_4.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_5.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_6.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_7.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_8.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_9.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_10.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_11.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2016_12.json
/content/drive/MyDrive/Data (1)/Weather_Data/SFO/2017_1.json
/content/drive/MyD

In [ ]:
# Concatenating the list of weather Data Frames
weather_data_df = pd.concat(weather_data)
weather_data_df = weather_data_df.reset_index(drop=True)
print(weather_data_df.info(), end="\n\n")
print(weather_data_df.columns, end="\n\n")
print(weather_data_df.shape, end="\n\n")
print(weather_data_df.index, end="\n\n")
# print(len(weather_data))
# Storing the combined data as a csv file
weather_data_df.to_csv("/content/drive/MyDrive/Data (1)/weather_data.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263160 entries, 0 to 263159
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   windspeedKmph  263160 non-null  object
 1   winddirDegree  263160 non-null  object
 2   weatherCode    263160 non-null  object
 3   precipMM       263160 non-null  object
 4   visibility     263160 non-null  object
 5   pressure       263160 non-null  object
 6   cloudcover     263160 non-null  object
 7   DewPointF      263160 non-null  object
 8   WindGustKmph   263160 non-null  object
 9   tempF          263160 non-null  object
 10  WindChillF     263160 non-null  object
 11  humidity       263160 non-null  object
 12  date           263160 non-null  object
 13  time           263160 non-null  object
 14  airport        263160 non-null  object
dtypes: object(15)
memory usage: 30.1+ MB
None

Index(['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pre

In [ ]:
#missing data
print(weather_data_df.isnull().sum())

windspeedKmph    0
winddirDegree    0
weatherCode      0
precipMM         0
visibility       0
pressure         0
cloudcover       0
DewPointF        0
WindGustKmph     0
tempF            0
WindChillF       0
humidity         0
date             0
time             0
airport          0
dtype: int64


In [ ]:
weather_data_df.head()

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL


In [ ]:
#handle duplicates
weather_data_df.drop_duplicates()
print(weather_data_df.shape, end="\n")

(263160, 15)


#Flight preprocessing


In [ ]:
import pandas as pd
import os
import missingno as msno
%matplotlib inline

# Processing the data only for the airports mentioned below
airports = ('ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO')
fields = ("Year",
          "Quarter",
          "Month",
          "DayofMonth",
          "FlightDate",
          "OriginAirportID",
          "Origin",
          "DestAirportID",
          "Dest",
          "CRSDepTime",
          "DepTime",
          "DepDelayMinutes",
          "DepDel15",
          "CRSArrTime",
          "ArrTime",
          "ArrDelayMinutes",
          "ArrDel15")


In [ ]:
# Loading only the selected columns
df = pd.read_csv("/content/drive/MyDrive/Data/Flight_Data/2016_1.csv", usecols=fields)
# Filtering out rows with valid origin and destination airports
df = df[df["Origin"].isin(airports)]
df = df[df["Dest"].isin(airports)]

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75010 entries, 57 to 445817
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             75010 non-null  int64  
 1   Quarter          75010 non-null  int64  
 2   Month            75010 non-null  int64  
 3   DayofMonth       75010 non-null  int64  
 4   FlightDate       75010 non-null  object 
 5   OriginAirportID  75010 non-null  int64  
 6   Origin           75010 non-null  object 
 7   DestAirportID    75010 non-null  int64  
 8   Dest             75010 non-null  object 
 9   CRSDepTime       75010 non-null  int64  
 10  DepTime          73195 non-null  float64
 11  DepDelayMinutes  73195 non-null  float64
 12  DepDel15         73195 non-null  float64
 13  CRSArrTime       75010 non-null  int64  
 14  ArrTime          73161 non-null  float64
 15  ArrDelayMinutes  73073 non-null  float64
 16  ArrDel15         73073 non-null  float64
dtypes: float64

In [ ]:
print(df.isnull().sum())

Year                  0
Quarter               0
Month                 0
DayofMonth            0
FlightDate            0
OriginAirportID       0
Origin                0
DestAirportID         0
Dest                  0
CRSDepTime            0
DepTime            1815
DepDelayMinutes    1815
DepDel15           1815
CRSArrTime            0
ArrTime            1849
ArrDelayMinutes    1937
ArrDel15           1937
dtype: int64


In [ ]:
df = df.dropna(how="any", axis=0)
print(df.isnull().sum())


Year               0
Quarter            0
Month              0
DayofMonth         0
FlightDate         0
OriginAirportID    0
Origin             0
DestAirportID      0
Dest               0
CRSDepTime         0
DepTime            0
DepDelayMinutes    0
DepDel15           0
CRSArrTime         0
ArrTime            0
ArrDelayMinutes    0
ArrDel15           0
dtype: int64


In [ ]:
# Loading all the csv files from the Data dir into a list with their relative path names
flight_data_path = "/content/drive/MyDrive/Data/Flight_Data"
flight_data_files = [os.path.join(flight_data_path, f) for f in os.listdir(flight_data_path)]
flight_data_files
flight_data = list()
# performing data handling
for file in flight_data_files:
    df = pd.read_csv(file, usecols=fields)
    df = df[df["Origin"].isin(airports)]
    df = df[df["Dest"].isin(airports)]
    df = df.dropna(how="any", axis=0)
    flight_data.append(df)
    print(file)

/content/drive/MyDrive/Data/Flight_Data/2016_1.csv
/content/drive/MyDrive/Data/Flight_Data/2016_2.csv
/content/drive/MyDrive/Data/Flight_Data/2016_3.csv
/content/drive/MyDrive/Data/Flight_Data/2016_4.csv
/content/drive/MyDrive/Data/Flight_Data/2016_5.csv
/content/drive/MyDrive/Data/Flight_Data/2016_6.csv
/content/drive/MyDrive/Data/Flight_Data/2016_7.csv
/content/drive/MyDrive/Data/Flight_Data/2016_8.csv
/content/drive/MyDrive/Data/Flight_Data/2016_9.csv
/content/drive/MyDrive/Data/Flight_Data/2016_10.csv
/content/drive/MyDrive/Data/Flight_Data/2016_11.csv
/content/drive/MyDrive/Data/Flight_Data/2016_12.csv
/content/drive/MyDrive/Data/Flight_Data/2017_7.csv
/content/drive/MyDrive/Data/Flight_Data/2017_6.csv
/content/drive/MyDrive/Data/Flight_Data/2017_8.csv
/content/drive/MyDrive/Data/Flight_Data/2017_9.csv
/content/drive/MyDrive/Data/Flight_Data/2017_10.csv
/content/drive/MyDrive/Data/Flight_Data/2017_11.csv
/content/drive/MyDrive/Data/Flight_Data/2017_12.csv
/content/drive/MyDrive/Da

In [ ]:
# Concatenating the list of flight Data Frames
flight_data_df = pd.concat(flight_data)
flight_data_df = flight_data_df.reset_index(drop=True)
print(flight_data_df.info())
# Storing the combined data as a csv file
flight_data_df.to_csv("/content/drive/MyDrive/Data/flight_data.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851436 entries, 0 to 1851435
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Year             int64  
 1   Quarter          int64  
 2   Month            int64  
 3   DayofMonth       int64  
 4   FlightDate       object 
 5   OriginAirportID  int64  
 6   Origin           object 
 7   DestAirportID    int64  
 8   Dest             object 
 9   CRSDepTime       int64  
 10  DepTime          float64
 11  DepDelayMinutes  float64
 12  DepDel15         float64
 13  CRSArrTime       int64  
 14  ArrTime          float64
 15  ArrDelayMinutes  float64
 16  ArrDel15         float64
dtypes: float64(6), int64(8), object(3)
memory usage: 240.1+ MB
None


In [ ]:
flight_data_df.head()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
0,2016,1,1,1,2016-01-01,14747,SEA,12478,JFK,745,741.0,0.0,0.0,1602,1610.0,8.0,0.0
1,2016,1,1,2,2016-01-02,14747,SEA,12478,JFK,745,737.0,0.0,0.0,1602,1613.0,11.0,0.0
2,2016,1,1,3,2016-01-03,14747,SEA,12478,JFK,745,743.0,0.0,0.0,1602,1547.0,0.0,0.0
3,2016,1,1,4,2016-01-04,14747,SEA,12478,JFK,745,737.0,0.0,0.0,1602,1551.0,0.0,0.0
4,2016,1,1,5,2016-01-05,14747,SEA,12478,JFK,710,708.0,0.0,0.0,1527,1524.0,0.0,0.0


In [ ]:
#Merging Flight and Weather data

#Merging

In [ ]:
# Loading the aggregated flight dataset and weather dataset
fd_df = pd.read_csv("/content/drive/MyDrive/Data/flight_data.csv", index_col=0)
wd_df = pd.read_csv("/content/drive/MyDrive/Data (1)/weather_data.csv", index_col=0)

In [ ]:
fd_df["CRSDepTime"]

0           745
1           745
2           745
3           745
4           710
           ... 
1851431    1320
1851432    1845
1851433     805
1851434    1150
1851435    1455
Name: CRSDepTime, Length: 1851436, dtype: int64

In [ ]:
wd_df["time"]

0            0
1          100
2          200
3          300
4          400
          ... 
263155    1900
263156    2000
263157    2100
263158    2200
263159    2300
Name: time, Length: 263160, dtype: int64

In [ ]:
# Creating a list to store the rounded CRSDepTime of flight data
new_time = []
# Iterating through CRSDepTime
for time in fd_df["CRSDepTime"]:
    time = int(time)
    # Obtaining the minutes
    m = time % 100
    # If it is less than 30 mins, just subtract to round down
    if m < 30:
        time = time - m
    # If it is greater than or equal to 30 mins, round up
    elif(m >= 30) :
        time = time + (100 - m)
    # If it is 2400, make it 0
    if(time == 2400):
        time = 0
    new_time.append(time)
fd_df["Time_new"] = new_time
fd_df[["Time_new", "CRSDepTime"]]

,Time_new,CRSDepTime
0,800,745
1,800,745
2,800,745
3,800,745
4,700,710
...,...,...
1851431,1300,1320
1851432,1900,1845
1851433,800,805
1851434,1200,1150


In [ ]:
#main merging part
#merging the data based on the departure airport, scheduled departure time and date
merged_df = pd.merge(fd_df, wd_df,
              how = "inner",
              left_on = ["Origin","Time_new", "FlightDate"],
              right_on = ["airport","time", "date"])
# Storing the merged dataset as a csv file
merged_df.to_csv("/content/drive/MyDrive/Data/flight_and_weather.csv")

In [ ]:
print(merged_df.isnull().sum())

Year               0
Quarter            0
Month              0
DayofMonth         0
FlightDate         0
OriginAirportID    0
Origin             0
DestAirportID      0
Dest               0
CRSDepTime         0
DepTime            0
DepDelayMinutes    0
DepDel15           0
CRSArrTime         0
ArrTime            0
ArrDelayMinutes    0
ArrDel15           0
Time_new           0
windspeedKmph      0
winddirDegree      0
weatherCode        0
precipMM           0
visibility         0
pressure           0
cloudcover         0
DewPointF          0
WindGustKmph       0
tempF              0
WindChillF         0
humidity           0
date               0
time               0
airport            0
dtype: int64


In [ ]:
print(merged_df.shape)

(1851436, 33)


In [ ]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1851436 entries, 0 to 1851435
Data columns (total 33 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Year             int64  
 1   Quarter          int64  
 2   Month            int64  
 3   DayofMonth       int64  
 4   FlightDate       object 
 5   OriginAirportID  int64  
 6   Origin           object 
 7   DestAirportID    int64  
 8   Dest             object 
 9   CRSDepTime       int64  
 10  DepTime          float64
 11  DepDelayMinutes  float64
 12  DepDel15         float64
 13  CRSArrTime       int64  
 14  ArrTime          float64
 15  ArrDelayMinutes  float64
 16  ArrDel15         float64
 17  Time_new         int64  
 18  windspeedKmph    int64  
 19  winddirDegree    int64  
 20  weatherCode      int64  
 21  precipMM         float64
 22  visibility       int64  
 23  pressure         int64  
 24  cloudcover       int64  
 25  DewPointF        int64  
 26  WindGustKmph     int64  
 27  tempF       

In [ ]:
merged_df.head()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,2016,1,1,1,2016-01-01,14747,SEA,12478,JFK,745,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
1,2016,1,1,1,2016-01-01,14747,SEA,14107,PHX,820,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
2,2016,1,1,1,2016-01-01,14747,SEA,11292,DEN,759,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
3,2016,1,1,1,2016-01-01,14747,SEA,12266,IAH,738,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
4,2016,1,1,1,2016-01-01,14747,SEA,10397,ATL,745,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA


In [ ]:
merged_df.drop_duplicates()
print(merged_df.shape)D

(1851436, 33)
